In [2]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer

from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [3]:
data = pd.read_csv("all-data.csv", names=['Label', 'Text'], encoding='latin-1')



In [4]:
data

,Label,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4846 non-null   object
 1   Text    4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


# Preprocessing

In [6]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    
    sequences = tokenizer.texts_to_sequences(texts)
    print("Vocab length:", len(tokenizer.word_index) + 1)
    
    max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    print("Maximum sequence length:", max_seq_length)
    
    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    
    return sequences




In [7]:
def preprocess_inputs(df):
    df = df.copy()
    
    sequences = get_sequences(df['Text'])
    
    label_mapping = {
        'negative': 0,
        'neutral': 1,
        'positive': 2,
    }
    
    y = df['Label'].replace(label_mapping)
    
    train_sequences, test_sequences, y_train, y_test = train_test_split(sequences, y, train_size=0.7, shuffle=True, random_state=1)
    
    return train_sequences, test_sequences, y_train, y_test

In [8]:
train_sequences, test_sequences, y_train, y_test = preprocess_inputs(data)

Vocab length: 10123
Maximum sequence length: 71


In [9]:
train_sequences

array([[5442,  510,   16, ...,    0,    0,    0],
       [  22, 1628,    4, ...,    0,    0,    0],
       [1141,  936,  136, ...,    0,    0,    0],
       ...,
       [   1,  419,   16, ...,    0,    0,    0],
       [2586,  123, 3247, ...,    0,    0,    0],
       [  30,  615,  555, ...,    0,    0,    0]])

In [10]:
y_train

545     2
2374    0
4217    1
1071    1
716     2
       ..
2895    1
2763    1
905     2
3980    1
235     2
Name: Label, Length: 3392, dtype: int64

# Training

In [11]:
inputs = tf.keras.Input(shape=(train_sequences.shape[1],))
x = tf.keras.layers.Embedding(
    input_dim=10123,
    output_dim=128,
    input_length=train_sequences.shape[1]
)(inputs)
x = tf.keras.layers.GRU(256, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_sequences,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
85/85 [==============================] - 12s 121ms/step - loss: 0.8283 - accuracy: 0.6421 - val_loss: 0.7895 - val_accuracy: 0.6480
Epoch 2/100
85/85 [==============================] - 10s 115ms/step - loss: 0.4787 - accuracy: 0.8150 - val_loss: 0.7499 - val_accuracy: 0.6657
Epoch 3/100
85/85 [==============================] - 10s 118ms/step - loss: 0.1632 - accuracy: 0.9451 - val_loss: 1.1227 - val_accuracy: 0.7084
Epoch 4/100
85/85 [==============================] - 10s 121ms/step - loss: 0.0414 - accuracy: 0.9853 - val_loss: 1.2382 - val_accuracy: 0.6951
Epoch 5/100
85/85 [==============================] - 10s 119ms/step - loss: 0.0237 - accuracy: 0.9948 - val_loss: 1.2112 - val_accuracy: 0.6686


In [12]:
results = model.evaluate(test_sequences, y_test, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))


    Test Loss: 0.70035
Test Accuracy: 70.63%


In [13]:

model.save('senti_model_main.h5')
